# Manipulations with .log files using Pandas library


In [12]:
import pandas as pd
import logging

To start, let's read .log file and parse it using pandas into a convenient table, and print out top 5 elements of the table using method .head()


In [13]:
NAME_OF_FILE = ""

In [14]:
def get_table_of_all_lines(file_name=NAME_OF_FILE):
    # Create a dataframe object and collect data from the log file
    data_lines = []
    with open(file_name, "r") as file:
        for line in file:
            data_lines.append(line.strip())

    return pd.DataFrame(data_lines, columns=["message"])

In [15]:
ORIGINAL_TABLE = get_table_of_all_lines()

top_5_elements = ORIGINAL_TABLE.head()
# print(top_5_elements)

However, we have a specific log formatting in the source code:


In [16]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
)

So in this case, our log convention is going to be

**time - level - message**

and we need a table, with these three columns. Let's make such a table.

Note:

- a line of the unknown format joins to the previous log-lines `"message"`.

- 'time' column elements become actual `datetime` objects


In [17]:
def get_table_of_logs(file_name=NAME_OF_FILE):
    data_lines = []
    split_sign = " - "
    how_many_columns = 3
    with open(file_name, "r") as file:
        for line in file:
            try:
                time, level, message = line.strip().split(split_sign, how_many_columns-1) # because start counting from 0
                data_lines.append({"time": time, "level": level, "message": message})
            except ValueError as e:
                # lines that do not fit our convention are joined with the last conventional log message we had
                data_lines[-1]["message"] += f"\n{line.strip()}"

    result = pd.DataFrame(data_lines)
    # parse time
    result["time"] = pd.to_datetime(result["time"], format="mixed")
    return result

In [18]:
LOGS_TABLE = get_table_of_logs()


top_5_elements = LOGS_TABLE.head()
# print(top_5_elements)

Now when we have a table, let's manipulate it in some way.

### Create a separate table

In [19]:
ERRORS_TABLE = LOGS_TABLE[LOGS_TABLE["level"] == "ERROR"]

top_5_elements = ERRORS_TABLE.head()
# print(top_5_elements)

sizes_of_the_error_table = ERRORS_TABLE.shape
# print(f"The number of rows: {sizes_of_the_error_table[0]}")
# print(f"The number of columns: {sizes_of_the_error_table[1]}")

The number of rows is the amount of errors.

### Count many stuff

In [20]:
# Count of unique error messages
unique_error_count = ERRORS_TABLE["message"].nunique()

# Count of all error messages (including duplicates)
all_error_count = sizes_of_the_error_table[0]  # number of ERRORS_TABLE rows

# For a detailed count of each unique error message
error_message_counts = ERRORS_TABLE["message"].value_counts() # it's a table unique_message - how_many

# print("Unique error messages count:", unique_error_count)
# print("Total error messages count:", all_error_count)
# print("\nCounts of each unique error message:\n", error_message_counts)

In [21]:
# Count of errors containing the word 'pending' in the message
pending_error_count = ERRORS_TABLE[
    ERRORS_TABLE["message"].str.contains("pending")
].shape[0]

# print("\nCounts of error messages that have word 'pending' in it :\n", pending_error_count)
first_error_time = ERRORS_TABLE["time"].iloc[0]
last_error_time = ERRORS_TABLE["time"].iloc[-1]

# print(f"The first error occurred at: {firt_error_time}")
# print(f"The last error occurred at: {last_error_time}")

Knowing how to count elements of the table, or to filter a table with respect to some criteria, you can create any statistics report you want.

### Let's save results

In [22]:
# Save the counts
my_statistics = {
    "log file name": NAME_OF_FILE,
    "first error time": first_error_time,
    "last error date": last_error_time,
    "unique error messages": unique_error_count,
    "total error messages": all_error_count,
    "pending error messages": pending_error_count,
}


def save_statistics(my_statistics=my_statistics):
    column1 = my_statistics.keys()
    column2 = my_statistics.values()

    counts_df = pd.DataFrame(data=column2, index=column1)

    counts_df.to_excel("error_counts.xlsx", header=False)


# save_statistics()

You can also save to csv, json, and txt (txt is going to be manually though), and [many other formats](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

See documentation of DataFrame object for further details.